# Knn

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler,MaxAbsScaler,RobustScaler,PowerTransformer,QuantileTransformer,Normalizer 
df = pd.read_csv("dataset/CL_underover-dataset.csv",sep='\t',decimal=",",index_col=0)

In [88]:
col_names = ['Imax','E','PIL', 'Mt', 'TsW', 'NbxW','MeanD','Eday_of_week']
features = df[col_names]
scaler = PowerTransformer()
X = scaler.fit_transform(features.values)

X_complete=df.copy()# Contiene oltre agli attributi per il clustering anche il custemerID
X_complete[col_names] = X
df=X_complete

In [89]:
df.head()

,Imax,E,PIL,Mt,TsW,NbxW,MeanD,Eday_of_week,class
0,0.763700,0.086062,0.12206,-0.046714,0.309153,0.834429,0.584471,1.266380,0
1,-1.890600,-1.288535,0.12206,-0.206633,-1.658318,-1.037838,-1.246122,-1.128287,0
2,-0.565636,-0.772488,0.12206,-0.395414,-1.135137,-1.037838,-1.246122,-1.128287,0
3,-0.565636,0.715159,0.12206,-0.137365,0.726504,1.688155,0.601997,1.170115,0
4,-0.812476,-0.594662,0.12206,0.060708,-0.344451,0.516536,0.809086,1.306169,0


In [90]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

label = df.pop('class')
train_set, test_set, train_label, test_label = train_test_split(df, label, stratify =label, test_size=0.20)

In [91]:
from sklearn.neighbors import KNeighborsClassifier

## Grid search

In [92]:
from sklearn.model_selection import GridSearchCV
grid_params={
    'n_neighbors':[3,5,10,15,20,50,100,200],
    'p':[1,2,5,10,20],
    'weights':['uniform','distance'],
    'algorithm':['auto'],
    'metric':['minkowski']
}

gs=GridSearchCV(KNeighborsClassifier(),grid_params,verbose=1,n_jobs=3,scoring='accuracy')
gs.fit(train_set,train_label)
print("Best Score: ",gs.best_score_)
print("Best Params: ",gs.best_params_)

print("Complete Grid search:")
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))


Fitting 5 folds for each of 80 candidates, totalling 400 fits
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.2s
[Parallel(n_jobs=3)]: Done 311 tasks      | elapsed:   31.7s
Best Score:  0.8860102601574036
Best Params:  {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
Complete Grid search:
0.862 (+/-0.022) for {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}
0.872 (+/-0.022) for {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
0.849 (+/-0.028) for {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
0.860 (+/-0.021) for {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
0.838 (+/-0.017) for {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 3, 'p': 5, 'weights': 'uniform'}
0.848 

## Modello finale

In [93]:
knn = KNeighborsClassifier(n_neighbors=15, weights='distance', metric='minkowski',p=1,algorithm='auto').fit(train_set, train_label)

In [94]:
#prediction on the test test
test_pred_knn = knn.predict(test_set)

In [95]:
#compute the performance of the model
print(classification_report(test_label, test_pred_knn))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92       338
           1       0.86      0.88      0.87       338
           2       0.92      0.96      0.94       338

    accuracy                           0.91      1014
   macro avg       0.91      0.91      0.91      1014
weighted avg       0.91      0.91      0.91      1014

